# The Subjectivity of Skin Tone: Evaluating Patient and Annotator Agreement Across Scales

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import re

# 1. Load the Data 

## A) Load Annotator Data

In [ ]:
# Load annotator data from Excel (all sheets)
def load_annotator_data(file_path):
    all_sheets_data = pd.read_excel(file_path, sheet_name=None)
    consolidated_df = pd.DataFrame()

    for sheet_name, sheet_data in all_sheets_data.items():
        annotator = sheet_name.split(' ')[1].strip()
        scale = sheet_name.split(' ')[2].strip()
        # Rename for homogenaity
        if scale == 'Fitz':
            scale = 'Fitzpatrick'

        sheet_data['Annotator'] = annotator
        sheet_data['Scale'] = scale
        # Rename the 'Confidence Score' column to include the scale name
        sheet_data.rename(columns={'Confidence Score': f'{scale} Confidence'}, inplace=True)
        consolidated_df = pd.concat([consolidated_df, sheet_data], ignore_index=True)
    
    return consolidated_df

In [ ]:
your_file_path = 'replace_with_your_data_path_here'

annotator_data = load_annotator_data(your_file_path)

print(annotator_data.head())

In [ ]:
def consolidate_annotator_data(df):
    # Extract unique image name (last part after "/") and parse Subject ID and Location
    df['Image_ID'] = df['Image Name'].apply(lambda x: x.split('/')[-1])
    df['Subject_ID'] = df['Image_ID'].apply(lambda x: re.search(r'\d+', x.split('_')[0]).group() if re.search(r'\d+', x.split('_')[0]) else None)
    
    def extract_location(image_id):
        if 'forehead' in image_id:
            return 'forehead'
        elif 'right_cheek' in image_id:
            return 'right cheek'
        elif 'left_cheek' in image_id:
            return 'left cheek'
        return 'unknown'  # If location is unspecified, label it as 'unknown'
    
    df['Location'] = df['Image_ID'].apply(extract_location)
    
    # Filter out rows where 'Subject_ID' contains scale names (like 'fenty', 'monk', 'fitz') -- the scale images need to be excluded
    df = df[~df['Subject_ID'].isin(['fenty', 'monk', 'fitz'])]

    # Initialize an empty list to collect the consolidated rows
    consolidated_data = []

    # Group by unique image and Subject_ID to consolidate entries
    for (image_id, subject_id, location), group in df.groupby(['Image_ID', 'Subject_ID', 'Location']):
        # Create a dictionary to store consolidated data for the image
        consolidated_entry = {
            'Image_ID': image_id,
            'Subject_ID': subject_id,
            'Location': location,
            'Image Name': group['Image Name'].iloc[0],  # Use the first instance
            'Timestamp': group['Timestamp'].iloc[0]      # Use the first timestamp instance
        }
        
        # Loop over each annotator's entry in the group
        for _, row in group.iterrows():
            scale = row['Scale']
            annotator = row['Annotator']
            
            # Add scale-specific scores and confidence with annotator designation
            score_column = f'{scale} Score'
            confidence_column = f'{scale} Confidence'

            # Add scale-specific scores and confidence with annotator designation
            consolidated_entry[f'{scale}_Score_Annotator{annotator}'] = row[score_column]
            consolidated_entry[f'{scale}_Confidence_Annotator{annotator}'] = row[confidence_column]
        
        # Append the consolidated entry to the list
        consolidated_data.append(consolidated_entry)

    # Convert the list of dictionaries into a DataFrame
    consolidated_df = pd.DataFrame(consolidated_data)

    # Return the consolidated DataFrame with unique Image_IDs and Annotator data as separate columns
    return consolidated_df

In [ ]:
consolidated_annotator_data = consolidate_annotator_data(annotator_data)
print(consolidated_annotator_data.head())
# print the size of the df
print(consolidated_annotator_data.shape)

## B) Load Patient Data

In [ ]:
# Load demographic data and standardize columns
def load_demographic_data(file_path):
    demographic_data = pd.read_excel(file_path)
    demographic_data.columns = demographic_data.columns.str.strip()
    # Replace Subject Number with Subject ID
    demographic_data.rename(columns={'Subject Number': 'Subject_ID'}, inplace=True)
    return demographic_data

In [ ]:
demographic_data = load_demographic_data('data/Patient_Demographics.xlsx')

print(demographic_data)

## C) Set Global Vars 
- Color Palette Variables: These hex values were extracted using this link (https://imagecolorpicker.com/) and the images used were the offical Monk Scale on Google's website (https://skintone.google/) and the fitzpatrick scale from this paper (https://pmc.ncbi.nlm.nih.gov/articles/PMC10566767/#:~:text=&text=The%20Fitzpatrick%20scale%20classifies%20skin,6%20as%20the%20darkest%20shade).

In [ ]:
fitzpatrick_palette = {
    "1.0": "#c6b49d", 
    "2.0": "#bea48b",  
    "3.0": "#af9479", 
    "4.0": "#a58367",  
    "5.0": "#896952",  
    "6.0": "#675043" 
}

monk_palette = {
    "1.0": "#f5ede4",
    "2.0": "#f3e7db",
    "3.0": "#f7eacf",
    "4.0": "#eadabb",
    "5.0": "#d7bd96",
    "6.0": "#a07e56",
    "7.0": "#825c43", 
    "8.0": "#604134",
    "9.0": "#3a312a",
    "10.0": "#2a2520"
}

# 2. Data Preprocessing & Merging 

## A) Helper Functions
- Roman Numeral Converter 
- Categorizing Races

In [ ]:
def roman_to_int(roman):
    if pd.isna(roman):
        return None
    elif isinstance(roman, float):
        return roman
    roman_map = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6}
    return roman_map.get(roman.upper().strip(), None)

def categorize_race_ethnicity(race_ethnicity):
    if pd.isna(race_ethnicity):
        return 'Unknown'
    race_ethnicity = race_ethnicity.lower().strip()
    categories = {
        'hispanic/latino': 'Hispanic/Latino',
        'white/white(not hispanic or latino)': 'White',
        'african american/black': 'African American/Black',
        'native american': 'Native American/Alaska Native',
        'native hawaiian/pacific islander': 'Native Hawaiian/Pacific Islander',
        'asian': 'Asian',
        'declined': 'Unknown',
        'other': 'Other'
    }
    for key, val in categories.items():
        if key in race_ethnicity:
            return val
    return 'Multiracial' if any(x in race_ethnicity for x in ['\\n', '&', 'and']) else 'Other'


## B) Merge Annotator and Patient Demographics 

In [ ]:
def merge_data(annotator_data, demographic_data):
    # Group Race and Ethnicity 
    demographic_data['Race (Grouped)'] = demographic_data['Race/Ethnicity'].apply(categorize_race_ethnicity)
    demographic_data['Fitzpatrick Score'] = demographic_data['Fitzpatrick Score'].apply(roman_to_int)
    
    # Strip whitespace in all string columns for both DataFrames
    annotator_data = annotator_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    demographic_data = demographic_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # make sure both subject IDs are strs
    annotator_data['Subject_ID'] = annotator_data['Subject_ID'].astype(str)
    demographic_data['Subject_ID'] = demographic_data['Subject_ID'].astype(str)
    
    # Merg the data via the Subject ID 
    merged_df = annotator_data.merge(demographic_data, on='Subject_ID', how='left')

    # drop any rows missing a subjective score 
    merged_df = merged_df.dropna(subset=['Fitzpatrick Score'])
    
    return merged_df

In [ ]:
master_df = merge_data(consolidated_annotator_data, demographic_data)
print(master_df.head())

# 3. Data Visualizations 

## A) Patient Demographics

In [ ]:
def plot_demographics(merged_df):
    # Keep only one entry per unique patient (based on Subject_ID)
    unique_patients_df = merged_df.drop_duplicates(subset=['Subject_ID'])

    #print the number of unique patients
    print(f'The number of unique patients is {unique_patients_df.shape[0]}')

    # Plot configurations as previously set up, but now using `unique_patients_df`
    plot_configs = [
        {"col": "Age", "kind": "hist", "title": "Age Distribution", "x_label": "Age", "kde": True, "bins": 10, "color": "blue"},
        {"col": "Sex", "kind": "count", "title": "Sex Distribution", "x_label": "Sex", "palette": "Set2"},
        {"col": "Race (Grouped)", "kind": "count", "title": "Race Distribution", "y_label": "Race", "palette": "Set3"},
        {"col": "Fitzpatrick Score", "kind": "hist", "title": "Fitzpatrick Score Distribution", "x_label": "Fitzpatrick Score", "kde": True, "bins": 6, "color": "grey"},
        {"col": "Monk Score", "kind": "hist", "title": "Monk Score Distribution", "x_label": "Monk Score", "kde": True, "bins": 10, "color": "grey"},
        {"col": "NATIVE LANGUAGE", "kind": "count", "title": "Native Language Distribution", "y_label": "Native Language", "palette": "Set1"},
        {"col": "Substance Use", "kind": "count", "title": "Substance Use Distribution", "y_label": "Substance Use", "palette": "Set2"}
    ]

    for config in plot_configs:
        plt.figure(figsize=(10, 6))
        if config["kind"] == "hist":
            sns.histplot(unique_patients_df[config["col"]], kde=config.get("kde", False), bins=config.get("bins", 10), color=config.get("color"))
        elif config["kind"] == "count":
            sns.countplot(y=config["col"], data=unique_patients_df, palette=config.get("palette"))
        
        plt.title(config["title"])
        plt.xlabel(config.get("x_label", "Count"))
        plt.ylabel(config.get("y_label", "Frequency"))
        plt.show()
    
        # Calculate the percentage of the cols unique values
        data = unique_patients_df[config["col"]]
        data = data.value_counts(normalize=True) * 100
        print(data)

        if config["col"] == 'Age': 
            # find the median age 
            median_age = unique_patients_df['Age'].median()
            print(f'The median age is {median_age}')
            # Find the IQR of the age
            Q1 = unique_patients_df['Age'].quantile(0.25)
            Q3 = unique_patients_df['Age'].quantile(0.75)
            IQR = Q3 - Q1
            print(f'The IQR of the age is {IQR}')

plot_demographics(master_df)

In [ ]:
# Demographics TABLE 
import pandas as pd
def generate_demographics_table(master_df):
    # Keep only unique patients
    unique_patients_df = master_df.drop_duplicates(subset=['Subject_ID']).copy()
    total_patients = unique_patients_df.shape[0]

    print(unique_patients_df.head())    

    # Helper function to group small categories into 'Other'
    def group_small_categories(series, threshold=0.05):
        value_counts = series.value_counts(normalize=True)
        small_categories = value_counts[value_counts < threshold].index
        return series.replace(small_categories, 'Other')

    # Apply CMS cell suppression for each column
    unique_patients_df['Sex'] = group_small_categories(unique_patients_df['Sex'])
    unique_patients_df['Race (Grouped)'] = group_small_categories(unique_patients_df['Race (Grouped)'])
    unique_patients_df['NATIVE LANGUAGE'] = group_small_categories(unique_patients_df['NATIVE LANGUAGE'])
    unique_patients_df['Substance Use'] = group_small_categories(unique_patients_df['Substance Use'])

    # Calculate demographic distributions
    demographics_table = {
        'Total Patients': total_patients,
        'Median Age': unique_patients_df['Age'].median(),
        'Age IQR': f"{unique_patients_df['Age'].quantile(0.25)} - {unique_patients_df['Age'].quantile(0.75)}",
        'Sex Distribution': unique_patients_df['Sex'].value_counts(normalize=True).map(lambda x: f"{x * 100:.2f}%").to_dict(),
        'Race Distribution': unique_patients_df['Race (Grouped)'].value_counts(normalize=True).map(lambda x: f"{x * 100:.2f}%").to_dict(),
        'Native Language Distribution': unique_patients_df['NATIVE LANGUAGE'].value_counts(normalize=True).map(lambda x: f"{x * 100:.2f}%").to_dict(),
        'Substance Use Distribution': unique_patients_df['Substance Use'].value_counts(normalize=True).map(lambda x: f"{x * 100:.2f}%").to_dict(),
        'Fitzpatrick Score Distribution': unique_patients_df['Fitzpatrick Score'].value_counts(normalize=True).map(lambda x: f"{x * 100:.2f}%").to_dict(),
        'Monk Score Distribution': unique_patients_df['Monk Score'].value_counts(normalize=True).map(lambda x: f"{x * 100:.2f}%").to_dict(),
    }

    # Flatten the nested dictionary for DataFrame conversion
    rows = []
    for key, value in demographics_table.items():
        if isinstance(value, dict):  # For distribution columns
            for sub_key, sub_value in value.items():
                rows.append({'Variable': key, 'Category': sub_key, 'Value': sub_value})
        else:
            rows.append({'Variable': key, 'Category': None, 'Value': value})

    formatted_table = pd.DataFrame(rows)
    return formatted_table

# Example usage
demographics_table = generate_demographics_table(master_df)

# Display the table in Jupyter Notebook
from IPython.display import display
display(demographics_table)

## B) Comparative Visualizations

In [ ]:
# 1. Fitzpatrick Score by Race/Ethnicity
# Keep only unique patients
unique_patients_df = master_df.drop_duplicates(subset=['Subject_ID']).copy()

# Helper function to group small categories into 'Other'
def group_small_categories(series, threshold=0.05):
    value_counts = series.value_counts(normalize=True)
    small_categories = value_counts[value_counts < threshold].index
    return series.replace(small_categories, 'Other')

# Apply CMS cell suppression for each column
unique_patients_df['Race (Grouped)'] = group_small_categories(unique_patients_df['Race (Grouped)'])

plt.figure(figsize=(10, 6))
sns.boxplot(x="Fitzpatrick Score", y="Race (Grouped)", data=unique_patients_df, palette="deep")
plt.title("Fitzpatrick Score by Self-Reported Race")
plt.xlabel("Fitzpatrick Score")
plt.ylabel("Self-Reported Race")
plt.show()

# 2. Monk Score by Race/Ethnicity
plt.figure(figsize=(10, 6))
sns.boxplot(x="Monk Score", y="Race (Grouped)", data=unique_patients_df, palette="deep")
plt.title("Monk Score by Self-Reported Race")
plt.xlabel("Monk Score")
plt.ylabel("Self-Reported Race")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_stacked_skin_tone_histogram(df, score_col, race_col='Race (Grouped)', 
                                     bins=10, palette='Set2', 
                                     title=None, x_label=None, y_label='Count'):
    """
    Plots a stacked histogram of the skin tone scores (e.g., Fitzpatrick or Monk) 
    with Race/Ethnicity groups stacked within each bin.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing at least the skin tone column and the race/ethnicity column.
    score_col : str
        The name of the column containing the skin tone scores (e.g. 'Fitzpatrick Score' or 'Monk Score').
    race_col : str
        The name of the column containing race/ethnicity group labels.
    bins : int
        Number of bins to use in the histogram.
    palette : str or list
        Seaborn color palette or list of colors.
    title : str
        Plot title.
    x_label : str
        X-axis label.
    y_label : str
        Y-axis label.
    """

    # Default labels if not provided
    if title is None:
        title = f"Distribution of {score_col} by {race_col}"
    if x_label is None:
        x_label = score_col

    plt.figure(figsize=(10, 6))
    
    # Create the stacked histogram
    ax = sns.histplot(
        data=df,
        x=score_col,
        hue=race_col,
        multiple='stack',
        bins=bins,
        palette=palette,
        edgecolor='black',
        legend='full'
    )

    # -- Force legend creation logic --
    # Try to grab the automatically-created legend
    auto_legend = ax.get_legend()

    if auto_legend is None:
        # If no legend was created automatically, create one manually
        handles, labels = ax.get_legend_handles_labels()
        # Sometimes the first label is just the name of the hue variable; remove if so
        if labels and labels[0] == race_col:
            handles, labels = handles[1:], labels[1:]
        # Only create a legend if we actually have more than 0 categories
        if labels:
            plt.legend(handles, labels, title=race_col, bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        # If a legend did get created, rename the title, move it, etc.
        auto_legend.set_title(race_col)
        auto_legend.set_bbox_to_anchor((1.05, 1))
        auto_legend.set_loc('upper left')

    # Aesthetics
    plt.title(title, fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.tight_layout()
    plt.show()


# NEW stacked histogram usage
plot_stacked_skin_tone_histogram(
    df=unique_patients_df, 
    score_col='Fitzpatrick Score', 
    race_col='Race (Grouped)', 
    bins=6, 
    palette='deep', 
    title='Stacked Histogram of Fitzpatrick Score by Self-Reported Race', 
    x_label='Fitzpatrick Score'
)

# If you also want a similar stacked histogram for Monk Score:
plot_stacked_skin_tone_histogram(
    df=unique_patients_df, 
    score_col='Monk Score', 
    race_col='Race (Grouped)', 
    bins=10, 
    palette='deep', 
    title='Stacked Histogram of Monk Score by Self-Reported Race', 
    x_label='Monk Score'
)

## C) Confidence Score Distributions 

In [ ]:
def plot_confidence_scores_histogram(merged_df, scales, annotators, num_bins=5, bin_range=(1, 5), palette="Set2"):
    # Define colors and labels using Seaborn palette
    colors = sns.color_palette(palette, len(annotators))
    labels = [f'Annotator {i}' for i in annotators]
    bar_width = 0.25
    x = np.arange(num_bins)  # Bin positions on the x-axis

    # Iterate over each scale
    for scale in scales:
        # Create a dictionary to hold the histogram counts for each annotator
        annotator_histograms = []
        
        # Calculate histogram counts for each annotator
        for annotator in annotators:
            confidence_column = f'{scale}_Confidence_Annotator{annotator}'
            if confidence_column in merged_df.columns:
                # Calculate histogram for the current annotator
                annotator_histogram, _ = np.histogram(merged_df[confidence_column].dropna(), bins=num_bins, range=bin_range)
                annotator_histograms.append(annotator_histogram)
        
        # Plot histograms for each annotator, side by side
        plt.figure(figsize=(12, 6))
        for i, (annotator_histogram, label, color) in enumerate(zip(annotator_histograms, labels, colors)):
            plt.bar(x + i * bar_width, annotator_histogram, width=bar_width, label=label, color=color)

        # Set plot labels and ticks
        plt.xlabel('Confidence Score')
        plt.ylabel('Frequency')
        plt.title(f'Confidence Scores for {scale} Scale by Annotator')
        plt.xticks(x + bar_width, range(1, num_bins + 1))  # Center x-ticks on bins
        plt.legend()
        plt.show()

# Define your scales and annotators
scales = ['Fitzpatrick', 'Monk']
annotators = [1, 2, 3]

# Example usage with Seaborn palette
plot_confidence_scores_histogram(master_df, scales, annotators, palette="Set2")




## D) Missing Data Analysis 

In [ ]:
# Missing Data Analysis 
def missingness_analysis(df):
    # Calculate the total and percentage of missing values for each column
    missing_data = df.isnull().sum()
    missing_percentage = (missing_data / len(df)) * 100
    
    # Create a summary DataFrame for missing data in the entire dataset
    missing_summary = pd.DataFrame({
        'Missing Values': missing_data,
        'Percentage': missing_percentage
    }).sort_values(by='Percentage', ascending=False)
    
    # Filter to include only columns with missing data
    missing_summary = missing_summary[missing_summary['Missing Values'] > 0]

    print("Missingness Analysis Summary for All Columns:")
    print(missing_summary)
    
    # Detailed analysis for critical columns 'Age' and 'Sex'
    missing_age = df[df['Age'].isnull()]
    missing_sex = df[df['Sex'].isnull()]
    missing_fitz = df[df['Fitzpatrick Score'].isnull()]
    missing_monk = df[df['Monk Score'].isnull()]
    
    if not missing_age.empty:
        print("\nDetailed Missingness for 'Age':")
        print("Number of missing 'Age' values:", len(missing_age))
        print("Missing 'Age' for Subject_IDs:")
        print(missing_age['Subject_ID'].unique())
    else:
        print("\nNo missing values found for 'Age'.")
    
    if not missing_sex.empty:
        print("\nDetailed Missingness for 'Sex':")
        print("Number of missing 'Sex' values:", len(missing_sex))
        print("Missing 'Sex' for Subject_IDs:")
        print(missing_sex['Subject_ID'].unique())
    else:
        print("\nNo missing values found for 'Sex'.")

    if not missing_fitz.empty:
        print("\nDetailed Missingness for 'Fitzpatrick Score':")
        print("Number of missing 'Fitzpatrick Score' values:", len(missing_fitz))
        print("Missing 'Fitzpatrick Score' for Subject_IDs:")
        print(missing_fitz['Subject_ID'].unique())
    else:
        print("\nNo missing values found for 'Fitzpatrick Score'.")
    
    if not missing_monk.empty:
        print("\nDetailed Missingness for 'Monk Score':")
        print("Number of missing 'Monk Score' values:", len(missing_monk))
        print("Missing 'Monk Score' for Subject_IDs:")
        print(missing_monk['Subject_ID'].unique())
    else:
        print("\nNo missing values found for 'Monk Score'.")
    
    return missing_summary

missing_data_summary = missingness_analysis(master_df)


# 4. Analysis 
- Intra Anatator
- Inter Rater 
- Logistic Regression 
- T-test 

## A) Intra-Annotator Analysis 
- Cronbach's Alpha with Bootstrapping for Confidence Intervals 

In [ ]:
from pingouin import cronbach_alpha

# Initialize results storage
cronbach_alpha_results = []

# Define scales and annotators
scales = ['Fitzpatrick', 'Monk']
annotators = [1, 2, 3]  # Adjust based on the number of annotators

# Bootstrapping function for confidence intervals
def bootstrap_cronbach_alpha(data, n_bootstrap=1000, alpha=0.05):
    """
    Perform bootstrapping to calculate confidence intervals for Cronbach's Alpha.
    
    Parameters:
        data (DataFrame): The dataset for Cronbach's Alpha calculation.
        n_bootstrap (int): Number of bootstrap iterations.
        alpha (float): Significance level for the confidence interval (default 0.05 for 95% CI).
    
    Returns:
        (float, float): Lower and upper bounds of the confidence interval.
    """
    alphas = []
    for _ in range(n_bootstrap):
        resample = data.sample(frac=1, replace=True)  # Resample with replacement
        alpha_val, _ = cronbach_alpha(resample)
        alphas.append(alpha_val)
    
    # Calculate the confidence interval
    lower = np.percentile(alphas, 100 * (alpha / 2))
    upper = np.percentile(alphas, 100 * (1 - alpha / 2))
    return lower, upper


# Loop through each scale and annotator, calculate Cronbach's Alpha, and store results
for scale in scales:
    for annotator in annotators:
        # Select the relevant score column for the current annotator and scale
        score_column = f'{scale}_Score_Annotator{annotator}'
        
        # Ensure the column exists in the DataFrame
        if score_column in master_df.columns:
            # Filter the DataFrame for the current scale and annotator
            scale_data = master_df[['Subject_ID', score_column]].dropna()
            
            # Group by Subject_ID and aggregate all images under each subject as separate columns
            subject_scores = scale_data.groupby('Subject_ID')[score_column].apply(lambda x: list(x)).to_frame()
            
            # Convert each list of scores into separate columns for each image within the subject
            subject_scores_expanded = pd.DataFrame(subject_scores[score_column].tolist(), index=subject_scores.index)
            subject_scores_expanded = subject_scores_expanded.dropna(axis=0)  # Drop rows with any missing values

            # Cronbach's Alpha calculation across all images (i.e., columns) for the subject
            if subject_scores_expanded.shape[1] > 1:  # Ensure at least 2 columns for Cronbach's Alpha
                alpha, _ = cronbach_alpha(subject_scores_expanded)
                # Perform bootstrapping for confidence intervals
                ci_lower, ci_upper = bootstrap_cronbach_alpha(subject_scores_expanded)
                
                interpretation = ('Excellent' if alpha >= 0.9 else
                                  'Acceptable' if alpha >= 0.8 else
                                  'Good' if alpha >= 0.7 else 'Poor')
                
                # Append the result for each annotator and scale
                cronbach_alpha_results.append({
                    'Annotator': annotator,
                    'Scale': scale,
                    'Cronbach\'s Alpha': round(alpha, 2),
                    '95% CI': f"( {round(ci_lower, 2)}, {round(ci_upper, 2)} )"
                    #'Interpretation': f'{interpretation} reliability'
                })
            else:
                print(f"Not enough data variation for Cronbach's Alpha calculation on {scale} - Annotator {annotator}.")

# Convert results to DataFrame and display
cronbach_alpha_results_df = pd.DataFrame(cronbach_alpha_results)
print(cronbach_alpha_results_df)

In [ ]:
# Make a Table of the Cronbach's Alpha Results without the Interpretation
#cronbach_alpha_results_df.drop(columns=['Interpretation'], inplace=True)

# Prepare the table data
data = cronbach_alpha_results_df.values
columns = cronbach_alpha_results_df.columns

# Create a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(8, 4))

# Hide the axes
ax.axis('tight')
ax.axis('off')

# Create a table
table = ax.table(
    cellText=data,
    colLabels=columns,
    loc="center",
    cellLoc="center",
)

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(columns))))

# Make the header row bold and give it a background color
for (row, col), cell in table.get_celld().items():
    if row == 0:  # Header row
        cell.set_fontsize(11)
        cell.set_text_props(weight="bold")  # Make text bold
        cell.set_facecolor("#f4f4f4")  # Light gray background
        cell.set_edgecolor("black")  # Black border
    elif row % 2 == 0:  # Alternating row colors
        cell.set_facecolor("#f9f9f9")  # Slightly lighter background
    else:
        cell.set_facecolor("white")  # Standard background

# Add a title above the table
plt.title(
    "Table X. Reliability Assessment Using Cronbach's Alpha",
    fontsize=12,
    weight="bold",
)
plt.subplots_adjust(top=1)

# Save as image or PDF
plt.savefig("styled_cronbach_alpha_table.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
# Annotator + Location Analysis
# Initialize results storage
cronbach_alpha_results = []

# Define scales and annotators
scales = ['Fitzpatrick', 'Monk']
annotators = [1, 2, 3]  # Adjust based on the number of annotators
locations = ['forehead', 'right cheek', 'left cheek']  # Predefined locations

# Bootstrapping function for confidence intervals
def bootstrap_cronbach_alpha(data, n_bootstrap=1000, alpha=0.05):
    """
    Perform bootstrapping to calculate confidence intervals for Cronbach's Alpha.
    """
    alphas = []
    for _ in range(n_bootstrap):
        resample = data.sample(frac=1, replace=True)  # Resample with replacement
        alpha_val, _ = cronbach_alpha(resample)
        alphas.append(alpha_val)
    
    # Calculate the confidence interval
    lower = np.percentile(alphas, 100 * (alpha / 2))
    upper = np.percentile(alphas, 100 * (1 - alpha / 2))
    return lower, upper

# Loop through each scale, annotator, and location
for scale in scales:
    for annotator in annotators:
        for location in locations:
            # Select the relevant score column for the current annotator and scale
            score_column = f'{scale}_Score_Annotator{annotator}'
            
            # Ensure the column exists in the DataFrame
            if score_column in master_df.columns:
                # Filter the DataFrame for the current scale, annotator, and location
                location_data = master_df[master_df['Location'] == location]
                scale_data = location_data[['Subject_ID', score_column]].dropna()
                
                # Group by Subject_ID and aggregate all images under each subject as separate columns
                subject_scores = scale_data.groupby('Subject_ID')[score_column].apply(lambda x: list(x)).to_frame()
                
                # Convert each list of scores into separate columns for each image within the subject
                subject_scores_expanded = pd.DataFrame(subject_scores[score_column].tolist(), index=subject_scores.index)
                subject_scores_expanded = subject_scores_expanded.dropna(axis=0)  # Drop rows with any missing values

                # Cronbach's Alpha calculation across all images (i.e., columns) for the subject
                if subject_scores_expanded.shape[1] > 1:  # Ensure at least 2 columns for Cronbach's Alpha
                    alpha, _ = cronbach_alpha(subject_scores_expanded)
                    # Perform bootstrapping for confidence intervals
                    ci_lower, ci_upper = bootstrap_cronbach_alpha(subject_scores_expanded)
                    
                    interpretation = ('Excellent' if alpha >= 0.9 else
                                      'Acceptable' if alpha >= 0.8 else
                                      'Good' if alpha >= 0.7 else 'Poor')
                    
                    # Append the result for each annotator, scale, and location
                    cronbach_alpha_results.append({
                        'Annotator': annotator,
                        'Scale': scale,
                        'Location': location,
                        'Cronbach\'s Alpha': round(alpha, 2),
                        '95% CI': f"( {round(ci_lower, 2)}, {round(ci_upper, 2)} )",
                        'Interpretation': f'{interpretation} reliability'
                    })
                else:
                    print(f"Not enough data variation for Cronbach's Alpha calculation on {scale} - Annotator {annotator} - Location {location}.")

# Convert results to DataFrame and display
cronbach_alpha_results_df = pd.DataFrame(cronbach_alpha_results)
print(cronbach_alpha_results_df)


In [ ]:
columns = ['Annotator', 'Scale', 'Location', 'Cronbach\'s Alpha', '95% CI']
df = cronbach_alpha_results_df
df = df.drop(columns=['Interpretation'])

# Sort by Location for grouping
df.sort_values(by=['Location','Scale'], inplace=True)

# Prepare the table data
table_data = df.values
table_columns = df.columns

# Create a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Hide the axes
ax.axis('tight')
ax.axis('off')

# Create a table
table = ax.table(
    cellText=table_data,
    colLabels=table_columns,
    loc="center",
    cellLoc="center",
)

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(10)
table.auto_set_column_width(col=list(range(len(table_columns))))

# Make the header row bold and give it a background color
for (row, col), cell in table.get_celld().items():
    if row == 0:  # Header row
        cell.set_fontsize(11)
        cell.set_text_props(weight="bold")  # Make text bold
        cell.set_facecolor("#f4f4f4")  # Light gray background
        cell.set_edgecolor("black")  # Black border
    elif row % 2 == 0:  # Alternating row colors
        cell.set_facecolor("#f9f9f9")  # Slightly lighter background
    else:
        cell.set_facecolor("white")  # Standard background

# Add a title above the table
plt.title(
    "Table X. Reliability Assessment Using Cronbach's Alpha by Location",
    fontsize=12,
    weight="bold",
)
plt.subplots_adjust(top=1)

# Save as image or PDF
plt.savefig("styled_cronbach_alpha_table_grouped_by_location.png", dpi=300, bbox_inches="tight")
plt.show()

## B) Inter-Annotator Analysis 
- Consensus Analysis versus Subjective: Box plot with Jitter, Violin, Scatter with Spearman, Bland-Altman
- Annotator Agreement: Weighted Cohen's Kappa, Kendall's W, Krippendorff 

### i) Consesus Analysis

In [ ]:
# Step 1: Calculate Mean Score Across Images for Each Annotator for Each Patient
def calculate_patient_level_consensus(master_df, scales, annotators):
    patient_level_data = []

    for scale in scales:
        for subject_id in master_df['Subject_ID'].unique():
            patient_data = master_df[master_df['Subject_ID'] == subject_id]
            consensus_scores = {}

            # Calculate the mean score for each annotator across images for this patient
            annotator_means = []
            for annotator in annotators:
                annotator_column = f'{scale}_Score_Annotator{annotator}'
                if annotator_column in patient_data.columns:
                    annotator_mean = patient_data[annotator_column].mean()
                    annotator_means.append(annotator_mean)
            
            # Calculate the consensus score as the mean across all annotator means
            consensus_score = sum(annotator_means) / len(annotator_means)
            
            # Get the subject's actual score for the scale
            subject_score = patient_data[f'{scale} Score'].iloc[0] if f'{scale} Score' in patient_data.columns else None

            # Add to patient-level data
            patient_level_data.append({
                'Subject_ID': subject_id,
                'Scale': scale,
                'Consensus_Score': consensus_score,
                'Subject_Score': subject_score
            })

    # Convert to DataFrame
    consensus_df = pd.DataFrame(patient_level_data)
    return consensus_df

# Example usage
scales = ['Fitzpatrick', 'Monk']
annotators = [1, 2, 3]
patient_level_consensus_df = calculate_patient_level_consensus(master_df, scales, annotators)
print(patient_level_consensus_df.head())


In [ ]:
# Step 2: Calculate Consensus vs Subjective Plots
import scipy.stats as stats

# 1. Scatter Plot with Spearman Correlation for Difference
def scatter_spearman_consensus(consensus_df, scale):
    consensus_df['Difference'] = consensus_df['Consensus_Score'] - consensus_df['Subject_Score']
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=consensus_df['Subject_Score'], y=consensus_df['Difference'])
    spearman_corr, _ = stats.spearmanr(consensus_df['Subject_Score'], consensus_df['Difference'], nan_policy='omit')
    plt.title(f'Spearman Correlation: {scale} Difference (Consensus - Subject) vs Subject Score')
    plt.xlabel('Subject Score')
    plt.ylabel('Difference (Consensus - Subject)')
    plt.text(0.7, 0.1, f'Spearman r={spearman_corr:.2f}', transform=plt.gca().transAxes)
    plt.show()

# 2. Box Plot with Jitter for Difference
def boxplot_jitter_consensus(consensus_df, scale, custom_palette):
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=consensus_df['Subject_Score'], y=consensus_df['Difference'], palette=custom_palette)
    sns.stripplot(x=consensus_df['Subject_Score'], y=consensus_df['Difference'], color='black', alpha=0.3)
    plt.title(f'{scale} Difference (Consensus - Subject) Distribution by Subject Score')
    plt.xlabel('Subject Score')
    plt.ylabel('Difference (Consensus - Subject)')
    plt.show()

# 3. Violin Plot for Difference -- with dynamic dashed lines 
# Function to check if color is too dark
def rgb2gray(r, g, b):
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray
def violin_plot_consensus(consensus_df, scale, custom_palette):
    plt.figure(figsize=(10, 6))
    # Extract hex values from the palette
    palette_colors = list(custom_palette.values())
    ax = sns.violinplot(x=consensus_df['Subject_Score'], y=consensus_df['Difference'], palette=custom_palette, inner="quartile")
    # Determine darkness of the palette colors (convert hex to RGB first)
    thresh = 0.6  # Threshold for darkness
    dark = [ rgb2gray(*mpl.colors.to_rgb(c)) < thresh for c in palette_colors]
        # Count the number of points for each category
    counts = consensus_df['Subject_Score'].value_counts().sort_index()
    # Track the index of the ax.lines to handle different line counts per violin
    line_index = 0
    for d, count in zip(dark, counts):
        # Each violin should ideally have three lines, but check if there are fewer
        if count > 1:
            # Only proceed if there are exactly three lines for this violin
            violin_lines = ax.lines[line_index:line_index + 3]
            line_index += 3  # Move to the next set of lines for the next violin
            
            # Apply color change if the palette color is dark
            if d:
                for line in violin_lines:
                    line.set_color('white')
        else:
            # Skip to the next violin without changing line color if only one point
            line_index += 1  # Only one line is present for single data points
    plt.title(f'Differences Between Annotator Consensus and Patient-Reported {scale} Scores Across Subject Scores')
    plt.xlabel('Subject Score')
    plt.ylabel('Difference (Consensus - Subject)')
    plt.show()

# 4. Bland-Altman Plot for Difference
def bland_altman_plot_consensus(consensus_df, scale):
    consensus = consensus_df['Consensus_Score']
    subject_score = consensus_df['Subject_Score']
    mean_score = (consensus + subject_score) / 2
    diff_score = consensus - subject_score
    plt.figure(figsize=(10, 6))
    plt.scatter(mean_score, diff_score, alpha=0.5)
    plt.axhline(diff_score.mean(), color='red', linestyle='--')
    plt.axhline(diff_score.mean() + 1.96 * diff_score.std(), color='blue', linestyle='--')
    plt.axhline(diff_score.mean() - 1.96 * diff_score.std(), color='blue', linestyle='--')
    plt.title(f'Bland-Altman Plot: {scale} Consensus vs Subject Score')
    plt.xlabel('Mean Score')
    plt.ylabel('Difference (Consensus - Subject)')
    plt.show()

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

def bland_altman_plot_consensus_2(consensus_df, scale):
    consensus = consensus_df['Consensus_Score']
    subject_score = consensus_df['Subject_Score']
    mean_score = (consensus + subject_score) / 2
    diff_score = consensus - subject_score
    
    # Compute Spearman's correlation between the mean and the difference
    spearman_corr, _ = stats.spearmanr(mean_score, diff_score, nan_policy='omit')

    # Create the Bland-Altman plot
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=mean_score, y=diff_score, alpha=0.5)

    # Calculate mean difference and limits of agreement
    mean_diff = diff_score.mean()
    std_diff = diff_score.std()
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff

    # Add horizontal lines for the mean and limits of agreement
    plt.axhline(mean_diff, color='red', linestyle='--', label=f'Mean Diff: {mean_diff:.2f}')
    plt.axhline(upper_limit, color='blue', linestyle='--', label=f'Upper Limit: {upper_limit:.2f}')
    plt.axhline(lower_limit, color='blue', linestyle='--', label=f'Lower Limit: {lower_limit:.2f}')

    # Fit and add a regression line (Loess alternative could be used for nonlinear trends)
    sns.regplot(x=mean_score, y=diff_score, scatter=False, color='black',
                line_kws={"linestyle": "dashed"}, label='Regression Line')

    # Add legend including Spearman correlation coefficient
    plt.legend(title=f'Spearman r: {spearman_corr:.2f}', loc='upper right')

    # Set plot title and labels
    plt.title(f'Bland-Altman Plot: {scale} Differences Between Annotator Consensus and Patient-Reported vs the Mean Score')
    plt.xlabel('Mean Score')
    plt.ylabel('Difference (Mean across Annotators - Self-Reported)')
    
    plt.show()

for scale in scales:
    # Filter DataFrame for the current scale
    consensus_df = patient_level_consensus_df[patient_level_consensus_df['Scale'] == scale]
    # Set custom color palette based on the scale
    if scale == 'Fitzpatrick':
        custom_palette = fitzpatrick_palette
    elif scale == 'Monk':
        custom_palette = monk_palette
    else: 
        custom_palette = "Set3"

    # Perform the analyses
    scatter_spearman_consensus(consensus_df, scale)
    boxplot_jitter_consensus(consensus_df, scale, custom_palette)
    violin_plot_consensus(consensus_df, scale, custom_palette)
    bland_altman_plot_consensus(consensus_df, scale)
    bland_altman_plot_consensus_2(consensus_df, scale)

### ii) Agreement Analysis

In [ ]:
# Step 1: Reorganize Master DF 

def prepare_data_with_all_ratings(master_df, scale, annotators, num_images=9):
    grouped_data = {}
    
    for subject_id in master_df['Subject_ID'].unique():
        subject_data = master_df[master_df['Subject_ID'] == subject_id]
        
        ratings = {'Subject_ID': subject_id}
        
        # Capture each annotator's scores across all images
        for annotator in annotators:
            for img_num in range(1, num_images + 1):
                score_col = f'{scale}_Score_Annotator{annotator}'
                img_col_name = f'{score_col}_Image{img_num}'
                
                # Extract rating if available
                if len(subject_data) >= img_num:
                    ratings[img_col_name] = subject_data[score_col].iloc[img_num - 1] if pd.notna(subject_data[score_col].iloc[img_num - 1]) else None
                else:
                    ratings[img_col_name] = None  # No rating for this image
            
        # Include the subject score if available
        ratings[f'{scale}_Subject_Score'] = subject_data[f'{scale} Score'].iloc[0] if f'{scale} Score' in subject_data.columns else None

        grouped_data[subject_id] = ratings

    grouped_df = pd.DataFrame.from_dict(grouped_data, orient='index').reset_index(drop=True)
    return grouped_df

def calculate_mean_scores(grouped_df, scale, annotators, num_images=9):
    # Calculate the mean score across all images for each patient and annotator
    mean_scores = {}
    for annotator in annotators:
        score_cols = [f'{scale}_Score_Annotator{annotator}_Image{i+1}' for i in range(num_images)]
        mean_scores[f'{scale}_Score_Annotator{annotator}'] = grouped_df[score_cols].mean(axis=1)
    
    # Combine into a single DataFrame with Subject_ID as the index
    mean_scores_df = pd.DataFrame(mean_scores)
    mean_scores_df['Subject_ID'] = grouped_df['Subject_ID']
    
    # Set Subject_ID as the index
    mean_scores_df = mean_scores_df.set_index('Subject_ID')
    return mean_scores_df

In [ ]:
# Table Maker Helper Function

def table_maker(data, columns, scale, title):
    # Create a Matplotlib figure and axis
    fig, ax = plt.subplots(figsize=(18, 5))

    # Hide the axes
    ax.axis('tight')
    ax.axis('off')

    # Create a table
    table = ax.table(
        cellText=data,
        colLabels=columns,
        loc="center",
        cellLoc="center",
    )

    # Style the table
    table.auto_set_font_size(False)
    table.set_fontsize(10)

    # Make the header row bold and give it a background color
    for (row, col), cell in table.get_celld().items():
        if row == 0:  # Header row
            cell.set_fontsize(11)
            cell.set_text_props(weight="bold")  # Make text bold
            cell.set_facecolor("#f4f4f4")  # Light gray background
            cell.set_edgecolor("black")  # Black border
        elif row % 2 == 0:  # Alternating row colors
            cell.set_facecolor("#f9f9f9")  # Slightly lighter background
        else:
            cell.set_facecolor("white")  # Standard background

    # Add a title above the table
    plt.title(
        title,
        fontsize=12,
        weight="bold",
    )
    plt.subplots_adjust(top=1)

    # Save as image or PDF
    plt.savefig(f"styled_icc_table_{scale}.png", dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
# Step 2: Calculate Inter-Annotator Agreement Stats
from pingouin import intraclass_corr, friedman
import krippendorff
from sklearn.metrics import cohen_kappa_score
import scipy.stats as stats

# Agreement Calculations
def agreement_measures_expanded(grouped_df, scale, annotators, num_images=9):
    # ICC Calculation: Prepare data for ICC by reshaping the DataFrame
    icc_data = pd.melt(
        grouped_df,
        id_vars=['Subject_ID'],
        value_vars=[col for col in grouped_df.columns if f'{scale}_Score_Annotator' in col],
        var_name='Rater_Image',
        value_name='Score'
    ).dropna()
    
    icc_data['Rater'] = icc_data['Rater_Image'].apply(lambda x: x.split('_')[2])  # Extract annotator ID
    icc_results = intraclass_corr(data=icc_data, targets='Subject_ID', raters='Rater', ratings='Score', nan_policy='omit').round(3)
    print(f"Intraclass Correlation (ICC) Results for {scale}:")
    print(icc_results)

    # Calculate pairwise Weighted Cohen's Kappa for each annotator pair across all images
    weighted_cohen_kappa_results = {}
    for i, annotator1 in enumerate(annotators):
        for annotator2 in annotators[i + 1:]:
            kappas = []
            for img_num in range(1, num_images + 1):
                score1 = grouped_df[f'{scale}_Score_Annotator{annotator1}_Image{img_num}'].dropna()
                score2 = grouped_df[f'{scale}_Score_Annotator{annotator2}_Image{img_num}'].dropna()
                if not score1.empty and not score2.empty:
                    kappa = cohen_kappa_score(score1.round(), score2.round(), weights='quadratic')
                    kappas.append(kappa)
            if kappas:
                avg_kappa = np.mean(kappas)
                print(f"Avg Weighted Cohen’s Kappa ({scale}): Annotator {annotator1} vs Annotator {annotator2} = {avg_kappa:.2f}")
                weighted_cohen_kappa_results[f"Annotator {annotator1} vs Annotator {annotator2}"] = [round(avg_kappa, 2)]
    print(weighted_cohen_kappa_results)
    return icc_results, pd.DataFrame(weighted_cohen_kappa_results, index=[0])

def calculate_kendalls_w_and_kripp_alpha(mean_scores_df, scale):
    # Print the data used for testing
    #print("Data for Kendall's W and Krippendorff's Alpha calculation:\n", mean_scores_df.head())

    # Kendall's W Calculation using friedman (from pingouin)
    kendall_w_value = None
    if not mean_scores_df.empty:
        friedman_result = friedman(mean_scores_df)
        kendall_w_value = friedman_result['W'].values[0]  # Extract Kendall's W
        print(f"Kendall's W for {scale}: {kendall_w_value:.2f}")

    # Krippendorff’s Alpha calculation
    kripp_alpha_value = krippendorff.alpha(mean_scores_df.to_numpy())
    print(f"Krippendorff's Alpha for {scale}: {kripp_alpha_value:.2f}")
    
    return kendall_w_value, kripp_alpha_value

# Example usage
scales = ['Fitzpatrick', 'Monk']
annotators = [1, 2, 3]
num_images = 9

inter_results = {f"Kendall's W": {}, "Krippendorff's Alpha": {}}
for scale in scales:
    grouped_df = prepare_data_with_all_ratings(master_df, scale, annotators, num_images)
    icc_results, weighted_ck = agreement_measures_expanded(grouped_df, scale, annotators)
    mean_scores_df = calculate_mean_scores(grouped_df, scale, annotators, num_images)
    
    # Calculate agreement metrics using mean scores
    kendallW, kripp_alpha = calculate_kendalls_w_and_kripp_alpha(mean_scores_df, scale)
    inter_results[f"Kendall's W"][scale] = round(kendallW, 2)
    inter_results[f"Krippendorff's Alpha"][scale] = round(kripp_alpha, 2) 

    # make a table for each of the results
    data = icc_results.values
    columns = icc_results.columns
    table_maker(data, columns, scale, f"Table X. Intraclass Correlation (ICC) Results for {scale}")

    data = weighted_ck.values
    columns = weighted_ck.columns
    table_maker(data, columns, scale, f"Table X. Weighted Cohen's Kappa Results for {scale}")

inter_results = pd.DataFrame(inter_results)
data = inter_results.values
columns = inter_results.columns
table_maker(data, columns, None, "Table X. Kendall's W and Krippendoff Values by scale")

## C) Linear Mixed Model Analysis

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

def elongate_data_for_model(master_df, scale):
    # Define columns for annotator scores and confidence levels
    annotator_score_cols = [f'{scale}_Score_Annotator{i}' for i in range(1, 4)]
    annotator_confidence_cols = [f'{scale}_Confidence_Annotator{i}' for i in range(1, 4)]
    
    # Create a long-format DataFrame for annotator-image combinations
    long_df = pd.DataFrame()
    
    for i, (score_col, conf_col) in enumerate(zip(annotator_score_cols, annotator_confidence_cols), start=1):
        annotator_df = master_df[['Image_ID', 'Subject_ID', 'Location', 'Timestamp', f'{scale} Score']].copy()
        annotator_df['Annotator'] = i
        annotator_df['Rating'] = master_df[score_col]
        annotator_df['Confidence'] = master_df[conf_col]
        annotator_df['Difference'] = annotator_df['Rating'] - annotator_df[f'{scale} Score']
        
        # Rename columns for consistency
        annotator_df = annotator_df.rename(columns={f'{scale} Score': 'SelfReportedScore', 'Location': 'Position', 'Subject_ID': 'PatientID'})
        
        # Append to long_df
        long_df = pd.concat([long_df, annotator_df], ignore_index=True)
    
    # Convert columns to categorical where appropriate
    long_df['Confidence'] = pd.Categorical(long_df['Confidence'], ordered=True)
    long_df['Annotator'] = long_df['Annotator'].astype('category')
    long_df['PatientID'] = long_df['PatientID'].astype('category')
    long_df['Position'] = long_df['Position'].astype('category')
    
    return long_df

def run_mixed_effects_model(master_df, scale):
    # Prepare the elongated data for the model
    df = elongate_data_for_model(master_df, scale)

    # Remove rows with missing values
    df = df.dropna()

    # drop the unique id column
    df = df.drop(columns=['Image_ID', 'Timestamp'])
    print(df.head())
    
    # Formula for the mixed-effects model
    formula = 'Difference ~ SelfReportedScore + Confidence + Position'
    
    # Specify the variance component for 'Annotator' as a random effect
    vc_formula = {'Annotator': '0 + C(Annotator)'}
    
    # Run the mixed-effects model using PatientID as the grouping variable (has duplicates??? )
    md = smf.mixedlm(formula, df, groups=df['PatientID'], vc_formula=vc_formula)
    mixed_model = md.fit()
    
    # Output results
    return mixed_model.summary()

# Example usage
for scale in ['Fitzpatrick', 'Monk']:
    print(f'{scale} Scale:')
    summary = run_mixed_effects_model(master_df, scale)
    print(summary)


## D) Paired T-Test
- The compares the mean annotators conseus as a group versus the patient's subjective score 

In [ ]:
from scipy import stats

def perform_t_test(mean_scores_df, scale, master_df):
    # get consensus scores
    mean_scores_df[f'Mean_Annotator_Score'] = mean_scores_df[[f'{scale}_Score_Annotator1', f'{scale}_Score_Annotator2', f'{scale}_Score_Annotator3']].mean(axis=1)
  
    # Get the subjective scores from the master_df
    subj_scores = master_df[['Subject_ID', f'{scale} Score']].drop_duplicates()
    
    # ALign by Subject ID
    subj_scores = subj_scores.reset_index()
    mean_scores_df = mean_scores_df.reset_index()

    # Merge the two dataframes
    merged_df = pd.merge(subj_scores, mean_scores_df, on='Subject_ID')

    # drop any rows with missing values
    #merged_df = merged_df.dropna()

    # Perform the paired t-test
    t_stat, p_val = stats.ttest_rel(merged_df['Mean_Annotator_Score'], merged_df[f'{scale} Score'])
        
    # Print and store results
    print(f'{scale} Scale - T-Statistic: {t_stat:.2f}, P-Value: {p_val:.3f}')
    return {'T-Statistic': t_stat, 'P-Value': p_val}


annotators = [1, 2, 3]
for scale in ['Fitzpatrick', 'Monk']:
    grouped_df = prepare_data_with_all_ratings(master_df, scale, annotators, 9)
    mean_scores_df = calculate_mean_scores(grouped_df, scale, annotators)

    # Perform the t-test
    results = perform_t_test(mean_scores_df, scale, master_df)
